## Efficiency Analysis of Non-Ideal Rankine Cycles 

First let's import some packages that will come in handy for our solution.  Note the two different packages from scipy for the interpolation, one for 1-D interpolation [docs](http://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html#id3) and one for 2-D interpolation [docs](http://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html#multivariate-data-interpolation-griddata) 

In [1]:
import numpy as np
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
%pylab inline --no-import-all


Populating the interactive namespace from numpy and matplotlib


### 1) Define our primary Data Structure for this program

In [2]:
def new_state(p=None,T=None,v=None,h=None,s=None,x=None,state=None):
    '''
    Create a new Thermodynamic State variable (dictionary), 
       using the values of any of the specified parameters
    
    Input Parameters
    -------------
    p:  Pressure (kPa)
    T:  Temperature (C)
    v:  Specific Volume (m^3/kg)
    h:  Enthalpy (kJ/kg)
    s:  Entropy (kJ/kg K)
    x:  Quality (value between 0 and 1)
    state:  a string describing the state. Either 'saturated' or 'superheated'
    
    Return value
    -----------
    a Dictionary containing the values of the specified parameters, or None for the unspecified parameters
    
    Example
    ----------
    State1 = new_state(p = 8000, h=1500) 
          --> {'s': None, 'p': 8000, 'v': None, 'T': None, 'state': None, 'h': 1500, 'x': None}
          
    '''
    
    return dict(p=p,T=T,v=v,h=h,s=s,state=state,x=x)

### 2) Define a Subroutine (function with no return value) to calculate the unknown values of the state properties

In [3]:
def calc_state(state):
    """
    Determine the thermodynamic properties of a in both the Saturated or Superheated regions
    
    Important!  This function is written to behave like a SUBROUTINE.  There is no return statement.
                The value of the input parameter "state" is modified to fill in any missing properties
    
    Input/Output Parameter
    ----------
    state : Dictionary that represents the thermodynamic state of the water/steam mixture. 
               This functions adds values for enthalpy (kJ/kg), entropy (kJ/(kg K)), 
               specific volume (m^3/kg), quality and temperature to "state"
               
            Important: The input state MUST have a specified PRESSURE (kPa) and at least one additional
                       specified paramater, selected from (x,s,h or v)
                       
                       
               
    Examples
    -------
    >>> State1 = new_state(p = 8000, x=1)
    >>> calc_state(State1)
    >>> print(State1)
              {'s': 5.7454897350293939, 'state': 'saturated', 'v': 0.023659805540533225,
              'T': 294.87194943981365, 'h': 2758.3216344152956, 'x': 1, 'p': 8000}
    >>> State1 = new_state(p = 8000, h=3500)
    >>> calc_state(State1)
    >>> print(State1)
              {'T': 541.4344262295082, 'h': 3500, 'state': 'superheated', 
              's': 6.8543032786885245, 'v': None, 'x': None, 'p': 8000}
    >>> State1 = new_state(p = 8000, h=1500)
    >>> calc_state(State1)
    >>> print(State1)
              {'T': 294.87194943981365, 'h': 1500.0, 'state': 'saturated', 's': 3.5295611828432571, 
              'v': 0.0042186322759568979, 'x': 0.12722139389359188, 'p': 8000}

    The text files:  sat_water_table.txt and superheated_water_table.txt are assumed 
    to be in the same folder as the ipynb file
    
    See Also
    --------
    CoolProp : http://www.coolprop.org
    """
        
    if state['p'] is None:                    # Pressure is required!! 
        raise NameError('Pressure was not specified')
    #endif:  state['p'] is None
    
    # extract the individual values from the state    
    pval=state['p'] / 100.0  #convert pressure from kPa to bar      
    xval=state['x']
    hval=state['h']
    sval=state['s'] 
    vval=state['v']
    tval=state['T']
    stateval=state['state']

    # are we KNOWN to be in the superheated region???
    if stateval =='superheated':  # Yes!  so handle the superheated region
                
        tcol, hcol,scol,pcol = np.loadtxt('superheated_water_table.txt', 
                                          skiprows = 1, unpack = True)
        
        pval=pval*100   # this table wants kPa ... undo the earlier conversion

        if tval is not None:   #Temperature is known, so calculate h, s and v
            state['h'] = float(griddata((tcol, pcol), hcol, (tval,pval)))
            state['s'] = float(griddata((tcol, pcol), scol, (tval,pval)))
        elif hval is not None:   #Temperature is known, so calculate h, s and v
            state['T'] = float(griddata((hcol, pcol), tcol, (hval,pval)))
            state['s'] = float(griddata((hcol, pcol), scol, (hval,pval)))
        elif sval is not None:   #Temperature is known, so calculate h, s and v
            state['T'] = float(griddata((scol, pcol), tcol, (sval,pval)))
            state['h'] = float(griddata((scol, pcol), hcol, (sval,pval)))
        else:
            raise NameError('Not enough properties specified in the superheated region') 
        #endif: 
        
        state['x']=None
        return None
    
    #endif:  ... are we superheated    
              
    # Assume we are in the saturated region ... until proven differently!
    
    # Read the saturated table data
    tcol, pcol, hfcol, hgcol, sfcol, sgcol, vfcol, vgcol = \
                np.loadtxt('sat_water_table.txt', skiprows = 1, unpack = True)
    
    # Using the known pressure, interpolate on the saturation tables columns
    # at the known pressure
    
    sf_func = interp1d(pcol, sfcol)  #this is the two line version of using interp1d
    sfval=sf_func(pval)
        
    sgval = interp1d(pcol, sgcol)(pval)  #this is the one line version of using interp1d
    vfval = interp1d(pcol, vfcol)(pval)
    vgval = interp1d(pcol, vgcol)(pval)
    hfval = interp1d(pcol, hfcol)(pval)
    hgval = interp1d(pcol, hgcol)(pval)
    tsat = interp1d(pcol, tcol)(pval)
            
    if xval is not None:  # x (quality) is known
        if xval<0  :
            raise NameError('Error - this function cannot operate in the sub-cooled region')
            
        elif xval > 1:     # now we know that we are in the SUPERHEATED region
            state['state']= 'superheated'
            calc_state(state)  #call calc_state to go back to the superheated code
            return None           
        else:              # now we know that we are in the SATURATED region
            state['h'] =  hfval + xval * (hgval-hfval)
            state['s'] =  sfval + xval * (sgval-sfval)
            state['v'] =  vfval + xval * (vgval-vfval) 
            state['T'] = float(tsat)
            state['state']='saturated'
            return None
        #endif:  x ...
        
    elif  hval is not None:
        state['x']= (hval-hfval)/(hgval-hfval)
        calc_state(state)
    elif  sval is not None:
        state['x']= (sval-sfval)/(sgval-sfval)
        calc_state(state)
    elif  vval is not None:
        state['x']= (vval-vfval)/(vgval-vfval)
        calc_state(state)
    elif tval is not None:  #pressure and temperature were given ... we might be superheated
        if tval > tsat:  #  we are in the superheated region
            state['state']='superheated'
            calc_state(state) #call calc_state to go back to the superheated code          
        elif  tval <= tsat:  # we are in the subcooled region
            raise NameError('This function cannot operate in the subcooled region')
        #endif
    else:
        raise NameError('Not enough properties specified')        
    #endif:  x is not None  


In [4]:
def print_state(state,name=None):
    if name is not None:
        print('State Name - {}'.format(name))
    if state['state'] is not None:
        print('{}'.format(state['state']))   
    if state['T'] is not None:
        print('T = {:.1f}'.format(state['T']))
    if state['p'] is not None:
        print('p = {:.2f}'.format(state['p']))
    if state['h'] is not None:
        print('h = {:.2f}'.format(state['h']))
    if state['s'] is not None:
        print('s = {:.5f}'.format(state['s']))
    if state['v'] is not None:
        print('v = {:.8f}'.format(state['v']))
    if state['x'] is not None:
        print('x = {:.5f}'.format(state['x']))
    print('')
    

In [5]:
# Test the calc_state function using the examples from the doc-string

State1 = new_state(p = 8000, x=1)
calc_state(State1)
print_state(State1,name='State1')

State2 = new_state(p = 8000, h=3500)
calc_state(State2)
print_state(State2,name='State2')

State3 = new_state(p = 8000, h=1500)
calc_state(State3)
print_state(State3)


State Name - State1
saturated
T = 294.9
p = 8000.00
h = 2758.32
s = 5.74549
v = 0.02365981
x = 1.00000

State Name - State2
superheated
T = 541.4
p = 8000.00
h = 3500.00
s = 6.85430

saturated
T = 294.9
p = 8000.00
h = 1500.00
s = 3.52956
v = 0.00421863
x = 0.12722



In [8]:
def Rankine_efficiency(p_high,p_mid,p_low,t_high,t_mid,turbine_efficiency,pump_efficiency):
    
    '''
    A function to calculate the cycle efficiency (expressed in %) of the Non-Ideal Rankine cycle.  
    
    Input Parameters
    ----------
    p_high:  the pressure at the turbine inlet, in units of kPa
    p_low:   the pressure at the turbine exit, in units of kPa
    t_high:  (optional)  The fluid temperature at the turbine inlet.  If given, 
                this indicates that the fluid at the turbine inlet is superheated.
                if omitted, then the fluid at the turbine inlet is saturated (x = 1)
    turbine_efficiency:  (optional)  a value between 0 and 1  (1 means 100% efficient)
    pump_efficiency:  (optional)  a value between 0 and 1 (1 means 100% efficient)
                       
    Return Value
    -----------
    The cycle efficiency expressed in percent (a value between 0% and 100%)
    
    Examples
    -----------
    Rankine_efficiency(8000,8)  -->  37.09
    Rankine_efficiency(6000,12) -->  34.84
    Rankine_efficiency(8000,8,turbine_efficiency=0.75, pump_efficiency=0.85) --> 27.70
    Rankine_efficiency(8000,8,t_high= 450,turbine_efficiency=0.75, pump_efficiency=0.85) -->29.38
    
    '''

    State1=new_state(p=p_high, T=t_high)  #Superheated
    calc_state(State1)  #turbine inlet

    State2s=new_state(p=p_mid,s=State1['s'])  # isentropic expansion
    calc_state(State2s)
    
    h2=State1['h']-turbine_efficiency*(State1['h']-State2s['h'])
    
    State2=new_state(p=p_mid,h=h2)
    calc_state(State2)

    State1b=new_state(p=p_mid, T=t_mid)  #Superheated
    calc_state(State1b)  #turbine inlet

    State2bs=new_state(p=p_low,s=State1b['s'])  # isentropic expansion
    calc_state(State2bs)
    
    h2b=State1b['h']-turbine_efficiency*(State1b['h']-State2bs['h'])
    
    State2b=new_state(p=p_low,h=h2b)
    calc_state(State2b)
    
    
    print_state(State1b,'1b')
    print_state(State2bs,'2bs')
    print_state(State2b,'2b')


    State3=new_state(p=p_low, x = 0)  # condense to pure liquid (x=0)
    calc_state(State3)

    h4s=State3['h']+State3['v']*(p_high - p_low)   # pump equation for enthalpy increase

    State4s=new_state(p=p_high,h=h4s)
    # Note:  State4 is in the subcooled region, so we can't use calc_state
    
    h4=State3['h']+1/pump_efficiency*(h4s-State3['h'])

    turbine_work=State1['h']-State2['h'] + State1b['h']-State2b['h']
    pump_work=h4-State3['h']
    heat_added = State1['h']-h4 + State1b['h']- State2['h']

    efficiency=(turbine_work-pump_work)/heat_added*100

    return efficiency


In [9]:
# Test the Rankine_efficiency function using the examples from the doc-string

e=Rankine_efficiency(8000,700,8,480,440,0.85, 0.9)
print('Rankine efficiency of system 3 is = {:.2f}'.format(e))

State Name - 1b
superheated
T = 440.0
p = 700.00
h = 3353.30
s = 7.76000

State Name - 2bs
saturated
T = 41.4
p = 8.00
h = 2428.21
s = 7.76000
v = 17.20182596
x = 0.93848

State Name - 2b
saturated
T = 41.4
p = 8.00
h = 2566.97
s = 8.20119
v = 18.26037139
x = 0.99623

Rankine efficiency of system 3 is = 35.09
